# Project 12: Spatial fairness in urban environments <br>

In [1]:
import geopandas as gpd
import folium
import numpy as np
import networkx as nx 
import pandas as pd
import itertools
import warnings
import sumolib
import os
#import openrouteservice as ors
from sklearn.preprocessing import MinMaxScaler

from shapely.geometry import LineString, Point

import warnings
warnings.filterwarnings("ignore")

from plot_utils import edges_to_gps_list, plot_paths, plot_bar_paths_cost
from routing_utils import from_sumo_to_igraph_network
from routing_algorithms import *
import matplotlib.pyplot as plt

attribute = 'traveltime'

### Utility <br>
Functions to improve readability

In [2]:
# modified version of the function of GiulianoCornacchia from the github,
# the only change is that the returned dataframe has one column for the edge id and one column for the geometry
def gdf_from_sumo_network(sumo_road_network, crs="EPSG:4326", edge_list=None):
    edges_to_plot = sumo_road_network.getEdges()
        
    list_line_strings = []
    list_ids = [] 

    for edge in edges_to_plot: 
        list_ids.append(edge.getID())      
        edge_gps_list = [sumo_road_network.convertXY2LonLat(x[0], x[1]) for x in edge.getShape()]
        list_line_strings.append(LineString(edge_gps_list))

    df_net = gpd.GeoDataFrame(data = list_ids, geometry=list_line_strings, crs="EPSG:4326") 

    df_net.columns = ['id', 'geometry']
    return df_net 

In [3]:
# function to print the correct row for the file xml
# path: tuple/list of edges
# i: identifier for the row
# type: type of trip, in this case if it is toward the city center or towards a random edge
# return: row ready to be written in the file
def path_to_xml(path, i, type):
    edges = ''
    depart = np.random.randint(0, 200)

    for e in path:
        edges+= str(e.getID()) + ' '

    s = '<vehicle id="'+type+str(i)+'" type="veh_passenger" depart="0" departLane="best"> <route edges="'+ edges +'"/> </vehicle> '
    return s

# function to print the correct row for the file xml
# path: list of edges ids
# i: identifier for the row
# type: type of trip, in this case if it is toward the city center or towards a random edge
# return: row ready to be written in the file
def list_to_xml(path, i, type):
    edges = ''
    depart = np.random.randint(0, 200)

    for e in path:
        edges+= str(e) + ' '

    s = '<vehicle id="'+type+str(i)+'" type="veh_passenger" depart="0" departLane="best"> <route edges="'+ edges +'"/> </vehicle> '
    return s

In [4]:
# create a blank map for the city of Leuven
def Leuven_map():
    point = (50.849738563227824, 4.644786576420597)
    m = folium.Map(location=point, zoom_start = 12)
    return m

m = Leuven_map()

The first step of the project was the data preparation. <br>
In this first part I opened three datasets:
* The real hub configuration, the locations of all mobility hubs in Leuven where mobility services are clustered together.
* The administrative zones of Leuven, GIS file with the sectors (identified by parameter CODSEC).
* The road network generated with SUMO

Then, some preliminary operations are performed. For each dataset I kept only the 'important' attributes, generally this means <br>
geometry, identifier and, in the case of the road network, the edge id.

## Data

In [5]:
# Open the hubs dataframe
url = 'https://storageaccount11111111.blob.core.windows.net/container1/Leuven/hub_data_leuven/mobility_hubs.gpkg'
hubs = gpd.read_file(url).to_crs('epsg:4326')[['nr', 'naam punt', 'geometry']]
folium.GeoJson(hubs).add_to(m)

In [6]:
# Open the zone dataframe
zones_leuven = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/leuven_statsec.gpkg')
zones_leuven = zones_leuven.to_crs('epsg:4326')[['CODSEC', 'geometry']]
folium.GeoJson(zones_leuven).add_to(m)

In [7]:
# road network from the sumo network
#
# build the network, the gdf and the graph directly from the sumo configuration
net = sumolib.net.readNet(os.getcwd()+'/Sumo/osm.net.xml')
road_net = gdf_from_sumo_network(net)
G = from_sumo_to_igraph_network(net)

Some operations were done on the dataframes.<br>

First, for each road it's useful to have its zone, so I performed a spatial join and retrieve only the street associated with a zone.<br>
For the pourpose of the project the dimension of the dataset it's enough that the simulation are statistically significant, but in <br>
future update one way is to infer new zones, *using clustering algorithm for example, and discard no streets.* <br>

Then, the opposite, it's easier to have only the zones with at least one represented road, so I decided to drop from the zones dataframe the ones without a road. <br>

In the hub dataframe, I decided to perform once again the spatial join, to retrieve its zone, and add the nearest edge in the SUMO net.<br>
In this way I can save computation time, since the possible starting points are aready computed. I decided to set a threshold in the <br>
searching of the nearest edge, and hubs with no edges in that radius are discarded from the dataset.

Lastly, I decided to extract, for each zone, its neighbors. Since one of the task is to retrieve the time needed for a user to exit from the neighborhood, <br>
it's easier to have a list of the adjecent zones ready at the use, and create flows between a zone and one of its neighbors.

In [8]:
road_net = road_net.sjoin(zones_leuven, how = 'inner')[['id', 'geometry', 'CODSEC']]

In [9]:
hubs = hubs.sjoin(zones_leuven, how = 'inner')[['nr', 'geometry', 'CODSEC']]
hubs.reset_index(inplace=True, drop = True)

In [10]:
#Popolate the geodataframe assigning, for each hub, its nearest edge in the SUMO network
nearest_edge = []

for idx, row in hubs.iterrows():
    hub = row['geometry']
    
    #nearest neighbouring edge of the hubs
    x_o, y_o = net.convertLonLat2XY(hub.x, hub.y)
    candidates_origin = net.getNeighboringEdges(x_o, y_o, r=35, includeJunctions=True)
    if len(candidates_origin) > 0:
        #origin edge
        nearest_edge.append(sorted(candidates_origin, key = lambda x: x[1])[0][0].getID())
    else:
        nearest_edge.append('NONE') #shouldn't happen, every hub should have a neighbouring edge

hubs['Edge'] = nearest_edge
# drop all the 'NONE', street, all the street without a zone
hubs.drop(list(np.where(hubs['Edge'] == 'NONE')[0]), axis = 0, inplace=True)

In [11]:
# in the same way, some of the zone may be useless, since they don't 'point' to any street
selected_zones = set(road_net['CODSEC'].tolist())
indexs_to_drop = []

for idx, row in zones_leuven.iterrows():
    if row['CODSEC'] not in selected_zones:
        indexs_to_drop.append(idx)

zones_leuven.drop(indexs_to_drop, inplace=True)
zones_leuven.reset_index(inplace=True, drop = True)

In [12]:
# retrieve adjacent zones, by zone
adjacent_zones = []

for idx, row in zones_leuven.iterrows():
    zones_row = []
    ids = set(np.where(zones_leuven['geometry'].intersection(zones_leuven['geometry'][idx]))[0]) - set([idx]) # remove the current zone from the list
    for id in ids:
        zones_row.append(zones_leuven['CODSEC'][id])

    adjacent_zones.append(zones_row)

zones_leuven['adjacent'] = adjacent_zones

In [13]:
# NORMALIZATION BY AREA
# area for zones in squared meters
utm = zones_leuven.estimate_utm_crs()
zones_leuven['area'] = zones_leuven.to_crs(utm).area

scaler = MinMaxScaler()
zones_leuven['area'] = scaler.fit_transform(np.array(zones_leuven['area']).reshape(-1,1))

# Create Traffic demand <br>

Now the traffic demand computation. In this part I wrote a function that, given a hub configuration, create a xml file compatible <br>
for SUMO to describe the flows in the network.

To meet the project goals, I needed to measure the level of segregation in the city, expressed as:
* the time needed by an user to getting out of a neighborhood 
* the time needed by an user to reach the city center


In this version the flows are assumed constant in dimension and their starting and ending point are drawn uniformely. **BASELINE**

In [16]:
# Create a traffic demand for the sumo network. 
# Starting from an hub configuration dataframe (points) compute two types of flows:
#   1) towards a random point outside its neighborhood (defined as one random edge in the adjacent zones)
#   2) towards the city center (defined ad-hoc)
# The process is repeated n (default = 100) times and in the end all the trips are written in the xml file in the Sumo folder.
# In this case, I decided to create flows of vehicles in an uniform way, each starting point is drawn at random in and uniform way.
# Currently, the paths are not computed, I decided to leave to duarouter the exact path computation. The number of vehicles in the flow
# is constant in this function. 
# @input hubs: geodataframe with the hubs (points) configuration
# @input fast: boolean value to decide if use shorted past or random graph
# @input n: number of trips to generate

def create_demand_uniform(hubs, n = 100):

    # city center information
    city_center = (4.70110, 50.87993)
    x_c, y_c = net.convertLonLat2XY(city_center[0], city_center[1])
    candidates_center = net.getNeighboringEdges(x_c, y_c, r=25, includeJunctions=True)
    e_center = sorted(candidates_center, key = lambda x: x[1])[0][0].getID()

    i = 0
    random_destination = ''
    center_destination = ''

    print('start generation phase...', end='')
    
    while i < n:
        #get a random mobility hub -- starting point
        random_hub = hubs.sample(1)       
        #get its zone
        start_zone = random_hub['CODSEC'].values[0]
        #nearest neighbouring edge of the origin
        origin = random_hub['Edge'].values[0]

        #select one zone of its neighbor zone
        neighbors = zones_leuven[zones_leuven['CODSEC'] == start_zone]['adjacent'].values[0]
        dest_zone = np.random.choice(neighbors)

        #destination point 1 -- random street in another zone
        destination = road_net[road_net['CODSEC'] == dest_zone].sample(1)['id'].values[0]
        random_destination += '<flow id = "r'+str(i)+'" begin="0" end = "500" departLane="random" number = "20" from = "'+origin+'" to = "'+destination+'" type="type1"></flow> \n'

        #destination point2 -- city center
        center_destination += '<flow id = "c'+str(i)+'" begin="0" end = "500" departLane="random" number = "20" from = "'+origin+'" to = "'+e_center+'" type="type1"></flow> \n'
        
        i +=1

    
    print('end')
    print('start writing file...', end='')
    # write the file
    f = open("Sumo/uniform_traffic_demand.xml", "w")
    f.write('<?xml version="1.0" encoding="UTF-8"?> <routes> <vType id="type1" length="5.00" maxSpeed="70.00" accel="2.6" decel="4.5" sigma="0.5" /> \n')
    f.write(random_destination)
    f.write(center_destination)
    f.write('</routes>')
    f.close()
    print('end')
    

## Duarouter

In [17]:
import subprocess

In [18]:
command_str = "duarouter --route-files Sumo/my_trips.xml -n Sumo/osm.net.xml " +\
                " --weights.random-factor 255 "+\
                "-o Sumo/traffic_demand_duarouter.rou.xml"

# SUMO

In [19]:
# import the traci library
import traci
import sys

In [20]:
# check sumo enviroment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

#Configuration 
sumo_binary = os.environ['SUMO_HOME']+"/bin/sumo"

#command for the simulation -- use duarouter
sumo_cmd = [sumo_binary, "-c", os.getcwd()+'/Sumo/my_config_duarouter.sumocfg', '-W', 'true', '--time-to-teleport', '5', '--no-step-log',
            '-v', 'false', '-t', 'false']

In [21]:
# Main loop 
# generate a traffic demand, call duarouter for things, run the simulation. Repeat the process n times and 
# compute the average times.
# @input: hubs, geodataframe with the hub configuration
# @output: random_average_time, average time, between all the simulation and all the different users, starting
#           from a random hub to exit from its neighborhood
# @output: center_average_time, average time, between all the simulation and all the different users, starting
#           from a random hub to reach the city center

def main_loop(hubs):
    random_average_time = []
    center_average_time = []

    #traci.vehicle.subsc

    for i in range(10):
        print('-------------')
        print('iteration ', i+1)

        #create random traffic demand
        create_demand_uniform(hubs, fast = True)

        duarouter
        print('start duarouter...', end= '')
        p = subprocess.Popen(command_str, shell=True, stdout=subprocess.PIPE, 
                                        stderr=subprocess.STDOUT)
        p.wait()
        print('end')

        #start new simulation
        print('start simulation...')
        traci.start(sumo_cmd)

        print('    Starting time: 0')
        random_time = dict()
        center_time = dict()

        #run simulation until no more vehicles
        while traci.simulation.getTime() < 500 or len(traci.vehicle.getIDList()) > 0 :
            traci.simulationStep()

            #get, for each vehicle, its last travel time, divided by destinaton
            for v in traci.vehicle.getIDList():
                distance = traci.vehicle.getDistance(v)
                time = traci.simulation.getTime()
                if distance == 0:
                    distance = 1

                timebydist = time/1
                if v[0] == 'r':
                    #normalize the time based on the distance
                    random_time[v] = timebydist
                else:
                    center_time[v] = timebydist

        print('    Ending time: ', traci.simulation.getTime())
        random_average_time.append(np.mean(list(random_time.values()))/60)
        center_average_time.append(np.mean(list(center_time.values()))/60)
        print('end')

        #close the simulation
        traci.close()
        #see if i can remove the output of traci
    return np.mean(random_average_time), np.mean(center_average_time)

## Real HUBS

In [22]:
random_mean, center_mean = main_loop(hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  924.0
end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  900.0
end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1227.0
end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  995.0
end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  925.0
end
-------------
iteration 

In [23]:
random_mean, center_mean

(4.6956621441783, 9.423441666666665)

## Random Hubs

In [24]:
# random hubs --> random street, centroid
np.random.seed(42)
r_hubs = road_net.sample(len(hubs), replace= True)

In [25]:
r_hubs.columns = ['Edge', 'geometry', 'CODSEC']

In [26]:
random_mean, center_mean = main_loop(r_hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1393.0
end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1439.0
end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1124.0
end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1394.0
end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1006.0
end
-------------
iterat

In [27]:
random_mean, center_mean

(5.560361307634165, 11.752374999999999)

## Optimal Placement

In [28]:
opt_hubs = gpd.read_file('opt_hubs.gpkg')[['geometry', 'index']]

In [29]:
opt_hubs = opt_hubs.sjoin(zones_leuven, how = 'inner')[['index', 'geometry', 'CODSEC']]
opt_hubs.reset_index(inplace=True, drop = True)

In [30]:
#Popolate the geodataframe assigning, for each street, its zone, SPATIAL JOIN
nearest_edge = []

for idx, row in opt_hubs.iterrows():
    hub = row['geometry']
    
    #nearest neighbouring edge of the opt_hubs
    x_o, y_o = net.convertLonLat2XY(hub.x, hub.y)
    candidates_origin = net.getNeighboringEdges(x_o, y_o, r=35, includeJunctions=True)
    if len(candidates_origin) > 0:
        #origin edge
        nearest_edge.append(sorted(candidates_origin, key = lambda x: x[1])[0][0].getID())
    else:
        nearest_edge.append('NONE') #shouldn't happen, every hub should have a neighbouring edge

opt_hubs['Edge'] = nearest_edge
# drop all the 'NONE', street, all the street without a zone
opt_hubs.drop(list(np.where(opt_hubs['Edge'] == 'NONE')[0]), axis = 0, inplace=True)

In [31]:
random_mean, center_mean = main_loop(opt_hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1361.0
end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1309.0
end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  866.0
end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  1300.0
end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...
 Retrying in 1 seconds
    Starting time: 0
    Ending time:  803.0
end
-------------
iteratio

In [32]:
random_mean, center_mean 

(5.271399723153483, 10.287146666666667)

# Traffic demand proportional to the flows

In [237]:
fdf = pd.read_csv('flussi_per_simulazione.csv')[['CODSEC_ORIG', 'CODSEC_DEST', 'flows_car_aggr', 'flows_pae_aggr']]

In [238]:
# starting point must have and hub
no_flow = []

for idx, row in fdf.iterrows():
    hub_list = hubs[hubs['CODSEC'] == row['CODSEC_ORIG']]
    if len(hub_list) == 0:
        no_flow.append(idx)

fdf.drop(no_flow, axis = 0, inplace= True)

In [239]:
sums = fdf[['flows_car_aggr', 'flows_pae_aggr']].sum(axis=0)
fdf['flows_car_aggr'] = fdf['flows_car_aggr']/sums['flows_car_aggr']
fdf['flows_pae_aggr'] = fdf['flows_pae_aggr']/sums['flows_pae_aggr']